> #### Adam Partl
> ## Bachelor thesis
> # Optimizing the automatic prediction of conversion to mild cognitive impairment using machine learning algorithms

### EDA

In [22]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

In [12]:
df = pd.read_csv('data/data.csv', low_memory=False)

Firstly we analyze the structure of the dataset

In [19]:
print(len(df.columns))
print(list(df.columns))

969
['NACCID', 'VISITMO', 'VISITDAY', 'VISITYR', 'NACCREFR', 'SEX', 'HISPANIC', 'HISPOR', 'HISPORX', 'RACE', 'RACEX', 'RACESEC', 'RACESECX', 'RACETER', 'RACETERX', 'PRIMLANG', 'PRIMLANX', 'EDUC', 'MARISTAT', 'NACCLIVS', 'INDEPEND', 'RESIDENC', 'HANDED', 'NEWINF', 'INHISPOR', 'INHISPOX', 'NACCNINR', 'INRACEX', 'INRASEC', 'INRASECX', 'INRATER', 'INRATERX', 'INRELTOX', 'INKNOWN', 'INVISITS', 'INCALLS', 'NACCFAM', 'NACCMOM', 'NACCDAD', 'NACCAM', 'NACCAMX', 'NACCAMS', 'NACCAMSX', 'NACCFM', 'NACCFMX', 'NACCFMS', 'NACCFMSX', 'NACCOM', 'NACCOMX', 'NACCOMS', 'NACCOMSX', 'NACCFADM', 'NACCFFTD', 'ANYMEDS', 'DRUG3', 'DRUG4', 'DRUG5', 'DRUG6', 'DRUG7', 'DRUG8', 'DRUG9', 'DRUG10', 'DRUG11', 'DRUG12', 'DRUG13', 'DRUG14', 'DRUG15', 'DRUG16', 'DRUG17', 'DRUG18', 'DRUG19', 'DRUG20', 'DRUG21', 'DRUG22', 'DRUG23', 'DRUG24', 'DRUG25', 'DRUG26', 'DRUG27', 'DRUG28', 'DRUG29', 'DRUG30', 'DRUG31', 'DRUG32', 'DRUG33', 'DRUG34', 'DRUG35', 'DRUG36', 'DRUG37', 'DRUG38', 'DRUG39', 'DRUG40', 'TOBAC30', 'TOBAC100', '

As can be seen, the dataset contains 969 columns 

In [20]:
print(df.describe())

            VISITMO      VISITDAY       VISITYR      NACCREFR           SEX  \
count  29673.000000  29673.000000  29673.000000  28889.000000  29673.000000   
mean       6.347555     15.546962   2013.687662      3.262834      1.611802   
std        3.409613      8.607724      6.098801      2.962000      0.487348   
min        1.000000      1.000000   2005.000000      1.000000      1.000000   
25%        3.000000      8.000000   2008.000000      1.000000      1.000000   
50%        6.000000     15.000000   2013.000000      2.000000      2.000000   
75%        9.000000     23.000000   2019.000000      8.000000      2.000000   
max       12.000000     31.000000   2024.000000      8.000000      2.000000   

           HISPANIC       HISPOR          RACE     RACESEC     RACETER  \
count  29541.000000  2752.000000  29458.000000  988.000000  201.000000   
mean       0.095799     4.986555      2.075599    8.094130    7.000000   
std        0.294320    10.269939      6.007969   15.400044   14.37

In [23]:
print(df.describe(exclude=np.number))

            NACCID HISPORX     RACEX  RACESECX RACETERX PRIMLANX INHISPOX  \
count        29673     132       445       116       18      401      113   
unique       29673      68       125        77       16      162       68   
top     NACC995870   Spain  HISPANIC  Hispanic  Mestizo   German    Spain   
freq             1      21        77         8        2       25       13   

         INRACEX INRASECX INRATERX INRELTOX NACCAMX NACCAMSX NACCFMX NACCFMSX  \
count        566      151       32      357    3132     3115    3116     3110   
unique       201       94       31      179      22       11      12        6   
top     HISPANIC   Jewish    Human     self       .        .       .        .   
freq          91       12        2       39    3105     3105    3103     3105   

       NACCOMX NACCOMSX                                DRUG3         DRUG4  \
count     3162     2172                                22292         19399   
unique      53       16                             

In [24]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29673 entries, 0 to 29672
Columns: 969 entries, NACCID to TIME
dtypes: float64(781), int64(53), object(135)
memory usage: 219.4+ MB
None


For the data description we are working with the NACC UDS Resarchers Data Dictionary (https://files.alz.washington.edu/documentation/uds3-rdd.pdf)
From the description of the data we can see the following:
> Cognitive Assessments
- NACCMMSE - Mini-Mental State Examination (MMSE) score
- NACCMOCA - Montreal Cognitive Assesment (MoCA) score
- MEMORY, ORIENT, JUDGMENT, CDRSUM - Cognitive function and dementia rating variables

> Neuropsychiatric Symptoms
- DEL, HALL, AGIT, DEPD, ANX, APA, DSIN - symptoms related to hallucinations, delusions, agitation, depression, anxiety, apathy, and disinhibition

> Family history and Genetic Risk
- NACCFAM - Family history of cognitive impairment
- NACCAM, NACCFADM - Alzheimer's related genetic mutations

> Functional Activities
- FAQ - BILLS, SHOPPING, STOVE, TRAVEL - decline in daily activities

> Neurological and Physical Exam Findings
- PARKSIGN, GAITDIS, RIGDL, BRADY - Parkinson signs and gait disorders
- CVDCOG, CBSTROKE, CORTDEF - Cerebrovascular disease and cortical deficits

> Health History and Risk Factors
- DIABETES, HYPERTEN, B12DEF, THYROID - common metabolic and vascular risk factors
- TBI - history of traumatic brain injury

> Clinician Judgment
- COGMEM, COGJUDG, COGLAND, COGATTN - Clinician-assessed impairment domains